 # Ejercicio 3
 ## Realizar pruebas unitarias para asegurar el correcto funcionamiento de cada módulo.


In [ ]:
import pandas as pd
import unittest

url = "https://demodata.grapecity.com/contoso/api/v1/employees"
json_name = "employees.json"

class Test(unittest.TestCase):
    def readApi(self):
        # Test para ver si lee de la api
        df = self.dowload_data(url,json_name)
        self.assertFalse(df.empty)


    def filtros(self):
        #Test para ver si aplica filtros
        df = self.get_current_employees("")
        self.assertTrue(df['isCurrent'].all())
        
        df = self.get_married_employees("")
        self.assertTrue((df['Married'] == True).all())

        df = self.get_last_name_start_with("", "S")
        self.assertTrue(df['LastName'].str.startswith("S").all())

    def exportar(self):
        #Test para ver si exporta a csv
        nombre_archivo = "employees.csv"
        try:
            df = self.exportar("https://demodata.grapecity.com/contoso/api/v1/employees", nombre_archivo)
            self.assertTrue(nombre_archivo.endswith(".csv"))
            df = pd.read_csv(nombre_archivo)
            self.assertIsInstance(df, pd.DataFrame)
        except Exception as e:
            self.fail(f"No se pudo leer el archivo CSV: {e}")


if __name__ == '__main__':
    unittest.main()

### Ejercicio 1
En primer lugar vamos a realizar una petición a la API para descargar la información necesaria.

In [7]:
import requests  #para realizar la solicitud
import json     #para leer el json
import pandas as pd #para trabajar con las bases de datos
# Dirección de la API
URL = "https://demodata.grapecity.com/contoso/api/v1/employees"
JSON_NAME = "employee_data.json"

ModuleNotFoundError: No module named 'requests'

In [ ]:

# Solicitamos la información mediante GET
def download_data(url, json_name):
    response = requests.get(url)

    # Comprobamos el estado de la solicitud y transformamos a json
    if response.status_code == 200:
        data = response.json()
        # Si no ha habido problema almacenamos el archivo json en local
        with open(json_name, "w") as json_file:
            json.dump(data, json_file, indent=4) #.dump pasa de objeto json a string.
        print(f"Archivo .json guardado correctamente en '{json_name}'" )
    else:
        print(f"Error al realizar la solicitud. HTTP Status Code: {response.status_code}")

download_data(URL, JSON_NAME)

<Response [200]>
Archivo .json guardado correctamente en employee_data.json


A continuación añadimos los siguientes filtros
- Seleccionar aquellos que están actualmente trabajando (`isCurrent =True`)
- Seleccionar aquellos que están casados (`maritalStatus = Married`)
- Seleccionar aquellos cuyo apellido empiece por una letra pasada por parámetros.

In [ ]:
#Incluimos la base de datos recién descargada
employee_db = pd.read_json("employee_data.json")
#employee_db.set_index(["employeeKey"])

,employeeKey,parentEmployeeKey,firstName,lastName,middleName,title,hireDate,birthDate,emailAddress,phone,...,isCurrent,isSalesPerson,departmentName,startDate,endDate,status,etlloadId,loadDate,updateDate,avatarUrl
0,1,18.0,Kieth,Moreno,None,Regional Manager,1996-07-31T00:00:00,1972-05-15T00:00:00,KiethMoreno@contoso.com,320-555-0195,...,True,False,Production,1996-07-31T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee1.jpg
1,2,18.0,Denis,Rivers,None,Regional Manager,1997-02-26T00:00:00,1977-06-03T00:00:00,DenisRivers@contoso.com,150-555-0189,...,True,False,Marketing,1997-02-26T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee2.jpg
2,3,18.0,Henry,Vane,None,Regional Manager,1997-12-12T00:00:00,1964-12-13T00:00:00,HenryVane@contoso.com,212-555-0187,...,True,False,Engineering,1997-12-12T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee3.jpg
3,4,18.0,Nathan,Wade,J.,Regional Manager,1998-01-05T00:00:00,1965-01-23T00:00:00,NathanWade@contoso.com,612-555-0100,...,True,False,Tool Design,1998-01-05T00:00:00,2000-06-30T00:00:00,NULL,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee4.jpg
4,5,18.0,Javier,Reid,None,Regional Manager,1998-01-05T00:00:00,1965-01-23T00:00:00,JavierReid@contoso.com,612-555-0100,...,True,False,Tool Design,2000-06-30T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,292,7.0,Logan,Bailey,None,Sales Store Manager,2002-07-01T00:00:00,1965-10-31T00:00:00,LoganBailey@contoso.com,1 (11) 500 555-0117,...,True,True,Sales,2002-07-01T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
289,293,7.0,Luke,May,None,Sales Store Manager,2002-11-01T00:00:00,1968-02-06T00:00:00,LukeMay@contoso.com,615-555-0153,...,True,True,Sales,2002-11-01T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
290,294,7.0,Nate,Brown,None,Sales Store Manager,2003-04-15T00:00:00,1965-02-11T00:00:00,NateBrown@contoso.com,926-555-0182,...,True,True,Sales,2003-04-15T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
291,295,7.0,Makenzie,Allen,None,Sales Store Manager,2003-07-01T00:00:00,1965-08-09T00:00:00,MakenzieAllen@contoso.com,1 (11) 500 555-0140,...,True,True,Sales,2003-07-01T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None


In [ ]:
def get_current_employees(db):
    return db[db["isCurrent"] == True]

def get_married_employees(db):
    return db[db["maritalStatus"].str.capitalize() == "Married"]

def get_last_name_start_with(db, letter):
    return db[db["lastName"].str.startswith(letter.upper())]

get_current_employees(employee_db)
get_married_employees(employee_db)
get_last_name_start_with(employee_db, "A")

,employeeKey,parentEmployeeKey,firstName,lastName,middleName,title,hireDate,birthDate,emailAddress,phone,...,isCurrent,isSalesPerson,departmentName,startDate,endDate,status,etlloadId,loadDate,updateDate,avatarUrl
36,37,6.0,Bart,Amstead,None,Sales Store Manager,1999-01-08T00:00:00,1967-04-19T00:00:00,BartAmstead@contoso.com,937-555-0137,...,True,False,Shipping and Receiving,1999-01-08T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee37.jpg
82,85,8.0,Percy,Ashwell,None,Sales Store Manager,1999-02-02T00:00:00,1968-07-01T00:00:00,PercyAshwell@contoso.com,172-555-0130,...,True,False,Production,1999-02-02T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee85.jpg
95,98,8.0,Tom,Addley,None,Sales Store Manager,1999-02-08T00:00:00,1971-12-08T00:00:00,TomAddley@contoso.com,148-555-0145,...,True,False,Production Control,1999-02-08T00:00:00,2001-10-01T00:00:00,NULL,1,2009-08-01T00:00:00,2009-08-01T00:00:00,/images/contoso/EmployeePhotos/Employee98.jpg
105,108,8.0,Kaylee,Abbey,None,Sales Store Manager,1999-02-13T00:00:00,1977-03-13T00:00:00,KayleeAbbey@contoso.com,476-555-0119,...,True,False,Production,1999-02-13T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
130,133,9.0,Ron,Appleton,None,Sales Store Manager,1999-02-25T00:00:00,1974-10-22T00:00:00,RonAppleton@contoso.com,935-555-0199,...,True,False,Finance,1999-02-25T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
157,160,9.0,Ethan,Addis,None,Sales Store Manager,1999-03-12T00:00:00,1966-03-13T00:00:00,EthanAddis@contoso.com,173-555-0179,...,True,False,Production,1999-03-12T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
159,162,9.0,Jayden,Attwood,None,Sales Store Manager,1999-03-12T00:00:00,1977-03-27T00:00:00,JaydenAttwood@contoso.com,181-555-0156,...,True,False,Research and Development,1999-03-12T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
169,172,9.0,Matthew,Attwood,None,Sales Store Manager,1999-03-17T00:00:00,1963-09-05T00:00:00,MatthewAttwood@contoso.com,238-555-0116,...,True,False,Production,1999-03-17T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
189,192,9.0,Erick,Atkinson,None,Sales Store Manager,1999-03-27T00:00:00,1970-06-29T00:00:00,ErickAtkinson@contoso.com,310-555-0133,...,True,False,Information Services,1999-03-27T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
228,231,10.0,Angelique,Avery,None,Sales Store Manager,2000-01-26T00:00:00,1976-10-20T00:00:00,AngeliqueAvery@contoso.com,283-555-0185,...,True,False,Production,2000-01-26T00:00:00,None,Current,1,2009-08-01T00:00:00,2009-08-01T00:00:00,None
